In [ ]:
import matplotlib.pyplot as plt
import numpy as np
import os

from preprocess_functions import preprocess #preprocess is a class containing all the function

## Set up paths and params, then load data and fill object
path is the root_data path as defined in https://github.com/ikharitonov/vestibular_vr_pipeline/issues/25)

Select sensors if sensor-specific (and not "auto") filtering is used. 'G8m', 'g5-HT3', 'rG1' or available sensors in the function, otherwise asks for user input for half decay time in ms.

Target area is the intended area, not verified by histology yet. Added to self.info dictionary.  

In [ ]:
path = '/Users/rancze/Documents/Data/vestVR/Cohort1/VestibularMismatch_day1/B6J2717-2024-12-12T12-54-49'
sensors = {'470':'g5-HT3', '560':'rG1', '410':'g5-HT3'}
target_area = ('X') 

In [ ]:
#Create an object which will contain an increasing amount of information as functions are called on
processed = preprocess(path, sensors)
# extract all relevant and irrelevant info from the Fluorescence.csv file which contains the metadata 
processed.info = processed.get_info()

In [ ]:
#Loads Events.csv and Fluorescence-unaligned.csv
#Aligns to 470 nm timestamps (assumes 470 exists) and cuts data if needed (almost never)
#Returns processed dataframes below
(
    processed.rawdata, 
    processed.data, 
    processed.data_seconds, 
    processed.signals, 
) = processed.create_basic(
    cutstart = False,
    cutend = False,
    target_area = target_area
)

In [ ]:
#Not sure why is even needed, not using currently, asked Hilde for clarification in issue #4
#processed.events = processed.extract_events()
'''
now we have an element 'events' containing timestamped events
for each event there will be a _starts and a _stops and a _event
 _starts: numpy nans for all rows except at the time stamp where the event starts
 _stops: numpy nans for all rows except at the time stamp where the event stops
 _events: False whenever the event did not take place, and True while it did take place
 The event is named the same as was as it was recorded
'''
#processed.events
None

### Filtering
All the sigals are low pass filtered using a butterworth filter.  
method = "auto" cutoff frequncy ~sample_rate/2 Hz  
method = "sensor" cutoff frequency is determined in the function using the sensors dictionary  
savefig = False by default, True will save the figure 

In [ ]:
processed.filtered = processed.low_pass_filt(method = "auto", plot=True, x_start=0, x_end=100) #for inspection
processed.filtered = processed.low_pass_filt(method = "auto", plot=True)

### Detrending
A double exponential fit is made to account for sources of bleaching and the signal is corrected.  
method = "subtractive" assumes bleaching is sensor-independent (e.g. autofluorescence)  
method = "divisive" assumes bleaching comes from the sensor. This is most plausible.  
savefig = False by default, True will save the figure  
**N.B.** divisive detrended data is already dF/F. 

In [ ]:
processed.data_detrended, processed.exp_fits = processed.detrend(plot = True, method = "divisive")

### Motion correction
There is a motion correction function that can be used. It is now set to use the 560 nm signal, because of my doubts with the relevans of the 410 nm signal as isosbestic trace. For now, I recommend not running this one.
Check function before use, not checked in Jan 2025. 

In [ ]:
#processed.motion_corr = processed.movement_correct(plot = True)

### Z-scoring
Standard Z-scoring of the signal.  
If divisive detrending was used, this is the z-scored dF/F.  
If subtractive detrending was used, this is the z-scored signal.  
motion = False does not use motion corrected signal  
savefig = False by default, True will save the figure

In [ ]:
processed.zscored = processed.z_score(motion = False, plot = True)

### Delta F / F

WITH divisive detrending, this is not needed

This is a standard way of calculating the detla F over F signal, i.e. the % change in signal. I do think it is a bit weird to use the detrending exponential fit again. I have wondered if I should change it to just a linear fit to the current detrended signal. For now I do this based on the fiber photometry primer paper code: https://github.com/ThomasAkam/photometry_preprocessing/blob/master/Photometry%20data%20preprocessing.ipynb

Again, 'motion' can be set to True, bu tis defaulth False
savefig = False by default, True will save the figure 

In [ ]:
processed.deltaF_F = processed.get_deltaF_F(motion = False, plot = True)

### Add crucial info
`processed.crucial_info = processed.add_crucial_info()`  
This has been removed and now added to the self.Info instead of columns in the saved fluorescence csv file

In [ ]:
# show structure of data frame to be saved 
processed.show_structure()

In [ ]:
processed.plot_all_signals()

### Save it as a .csv files
from Hilde  
This function will lead to it all being saved as a csv file which can easily be read as a pandas dataframe when the data is to be analysed.
First it is the info csv, which I for now save, but never actually use...
Then it is the main csv file which is very useful indeed. For this one you can add Events = True to also save the events, and motion_correct = True if you have doen motion correction and want to use this.The only difference for the latter, is really that it also saved the motion corrected raw signal. Regardless, if you did use motion correction for deltaF and z-score, this is the version that will be saved

In [ ]:
#again it ensures that the folder to save in already exists, since the csv must have somewhere to be
processed.info_csv = processed.write_info_csv()
processed.data_csv = processed.write_preprocessed_csv()
#optional:, motion_correct = True, Onix_align =False

### Have a look
By importing pandas, you can now read the file, by compying the path from above and adding 'preprocessed.csv' which is the name of your new file. Sorry about the unnamed file. It can be removed. I'll do that

In [ ]:
import pandas as pd
pd.read_csv(processed.save_path+'/Events.csv')

In [ ]:
pd.read_csv(processed.save_path+'/Processed_fluorescence.csv')